In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime

import tensorflow as tf
import numpy as np

from data import wider_face_single
from utils import intersection_over_union
import layers

In [3]:
# TensorBoard logging
ROOT_LOGDIR = "wider_single_logs"

def log_run():
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    logdir = "{}/run-{}".format(ROOT_LOGDIR, now)
    
    return logdir

In [4]:
# Stabilize ouptut accross runs
# TODO give the caller more control over what to reset

def reset(seed=42):
    tf.reset_default_graph()
    np.random.seed(seed)
    tf.set_random_seed(42)
    
reset()

In [5]:
IMG_SIZE = 64

In [6]:
num_train = 2992
num_val = 800
train, val, _ = wider_face_single.load_data(num_train=num_train, num_val=num_val, img_size=IMG_SIZE)

In [16]:
tf.reset_default_graph()

## Inputs
X = tf.placeholder(shape=[None, IMG_SIZE, IMG_SIZE, 3], dtype=tf.float32, name="X")
y = tf.placeholder(shape=[None, 4], dtype=tf.float32, name="y")

## Model
conv1 = tf.layers.conv2d(
    X, 
    filters=256,
    kernel_size=9,
    strides=1,
    padding="valid",
    name="conv1"
)

primaryCaps = layers.primaryCaps(
    conv1, 
    caps=32, 
    dims=8,
    kernel_size=9,
    strides=2,
    name="primaryCaps"
)

secondaryCaps = layers.denseCaps(
    primaryCaps, 
    caps=1, 
    dims=16,
    name="secondaryCaps"
)

dense_input = tf.reshape(
    secondaryCaps,
    [-1, 16],
    name='dense_input'
)

dense_1 = tf.layers.dense(
    dense_input,
    units=128,
    activation=tf.nn.relu,
    name='dense1'
)

dense_2 = tf.layers.dense(
    dense_1,
    units=128,
    activation=tf.nn.relu,
    name='dense2'
)

# Use ReLU nonlinearity in the last layer to force bboxes to have positive extent
predictions = tf.layers.dense(
    dense_2,
    units=4,
    name='predictions'
)

squared_errors = tf.square(
    y - predictions,
    name='squared_errors'
)

loss = tf.reduce_mean(squared_errors, name="loss")

train_loss_summary = tf.summary.scalar('train_loss', loss)
val_loss_summary = tf.summary.scalar('val_loss', loss)

accuracy = tf.reduce_mean(intersection_over_union(predictions, y), name='accuracy')

train_acc_summary = tf.summary.scalar('train_acc', accuracy)
val_acc_summary = tf.summary.scalar('val_acc', accuracy)

train_summaries = tf.summary.merge([train_loss_summary, train_acc_summary])
val_summaries = tf.summary.merge([val_loss_summary, val_acc_summary])

In [17]:
logdir = log_run()
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

optimizer = tf.train.AdamOptimizer(1e-3)
train_op = optimizer.minimize(loss, name='trainign_op')

init= tf.global_variables_initializer()

num_epochs = 50
batch_size = 20

train_iterations_per_epoch = num_train // batch_size
val_iterations_per_epoch = num_val // batch_size

In [76]:
# Overfit on num_samples
num_samples = 10

X_overfit, y_overfit = next(train.batch(num_samples))

with tf.Session() as sess:
    init.run()

    for epoch in range(num_epochs):
            
        train_op.run(
            feed_dict={
                X: X_overfit.reshape([-1, IMG_SIZE, IMG_SIZE, 3]),
                y: y_overfit.reshape([-1, 4])
            }
        )
            
        if (epoch + 1) % 10 == 0:
                
            train_loss, train_accuracy = sess.run(
                [loss, accuracy],
                feed_dict={
                    X: X_overfit.reshape([-1, IMG_SIZE, IMG_SIZE, 3]),
                    y: y_overfit.reshape([-1, 4])
                }
            )
        
            print('\rEpoch {}/{} ({:.1f}%) | IoU: {:.4f}% Loss: {:.6f}'.format(
                epoch + 1,
                num_epochs,
                (epoch + 1) * 100 / num_epochs,
                train_accuracy * 100,
                train_loss
            ))
            

Epoch 10/500 (2.0%) | IoU: 0.0000% Loss: 411.179108
Epoch 20/500 (4.0%) | IoU: 0.0000% Loss: 388.584045
Epoch 30/500 (6.0%) | IoU: 0.0000% Loss: 347.423981
Epoch 40/500 (8.0%) | IoU: 0.0670% Loss: 281.345703


KeyboardInterrupt: 

In [18]:
# Train the network

with tf.Session() as sess:
    init.run()
    
    for epoch in range(num_epochs):
        print('| [Epoch {}/{}'.format(epoch + 1, num_epochs))
        
        # TRAINING PHASE
        train_losses = []
        train_IoUs = []
        
        for iteration, (X_batch, y_batch) in enumerate(train.batch(batch_size)):
            
            train_op.run(
                feed_dict={
                    X: X_batch.reshape([-1, IMG_SIZE, IMG_SIZE, 3]),
                    y: y_batch.reshape([-1, 4])
                }
            )
            
            train_loss, train_IoU = sess.run(
                [loss, accuracy],
                feed_dict={
                    X: X_batch.reshape([-1, IMG_SIZE, IMG_SIZE, 3]),
                    y: y_batch.reshape([-1, 4]),
                }
            )
            
            train_losses.append(train_loss)
            train_IoUs.append(train_IoU)
            
            if iteration % 10 == 0:
                print(
                    '\r|>> Train | Iteration {}/{} ({:.1f})% | Batch IoU: {:.4f}% Loss: {:.6f}'.format(
                        iteration + 1,
                        train_iterations_per_epoch,
                        (iteration + 1) * 100 / train_iterations_per_epoch,
                        train_IoU,
                        train_loss
                    ),
                    end=''
                )
            
        epoch_train_loss = np.mean(train_losses)
        epoch_train_IoU = np.mean(train_IoUs)
        
        print(
            '\r|>> Train | Average IoU: {:.4f}% Loss: {:.6f}'.format(
                epoch_train_IoU * 100,
                epoch_train_loss
            ), 
            end=" " * 90 + "\n"
        )
        
        # VALIDATION PHASE
        val_losses = []
        val_IoUs = []
        
        for iteration, (X_batch, y_batch) in enumerate(val.batch(batch_size)):
            
            val_loss, val_IoU = sess.run(
                [loss, accuracy],
                feed_dict={
                    X: X_batch.reshape([-1, IMG_SIZE, IMG_SIZE, 3]),
                    y: y_batch.reshape([-1, 4]),
                }
            )
            
            val_losses.append(train_loss)
            val_IoUs.append(train_IoU)
            
            if iteration % 10 == 0:
                print(
                    '|>> Validation | Iteration {}/{} ({:.1f})% | Batch IoU: {:.4f}% Loss: {:.6f}'.format(
                        iteration + 1,
                        val_iterations_per_epoch,
                        (iteration + 1) * 100 / val_iterations_per_epoch,
                        val_IoU,
                        val_loss
                    ),
                    end=''
                )
            
        epoch_val_loss = np.mean(val_losses)
        epoch_val_IoU = np.mean(val_IoUs)
        
        print(
            '\r|>> Validation | Average IoU: {:.4f}% Loss: {:.6f}'.format(
                epoch_val_IoU * 100,
                epoch_val_loss
            ), 
            end=" " * 90 + "\n"
        )

| [Epoch 1/50
|>> Train | Average IoU: 8.2518% Loss: 198.800781                                                                                          
|>> Validation | Average IoU: 21.8394% Loss: 122.114868                                                                                          
| [Epoch 2/50
|>> Train | Average IoU: 13.6786% Loss: 115.492836                                                                                          
|>> Validation | Average IoU: 21.5696% Loss: 121.782616                                                                                          
| [Epoch 3/50
|>> Train | Average IoU: 13.6088% Loss: 115.440155                                                                                          
|>> Validation | Average IoU: 21.9136% Loss: 121.900978                                                                                          
| [Epoch 4/50
|>> Train | Average IoU: 13.8573% Loss: 114.601730                                  

|>> Train | Average IoU: 32.3845% Loss: 24.266787                                                                                          
|>> Validation | Average IoU: 47.2268% Loss: 12.160858                                                                                          
| [Epoch 29/50
|>> Train | Average IoU: 32.5253% Loss: 23.217314                                                                                          
|>> Validation | Average IoU: 45.2530% Loss: 15.255776                                                                                          
| [Epoch 30/50
|>> Train | Average IoU: 33.5590% Loss: 21.575220                                                                                          
|>> Validation | Average IoU: 52.9205% Loss: 13.553718                                                                                          
| [Epoch 31/50
|>> Train | Average IoU: 32.5768% Loss: 22.300232                                                   